In [3]:
import gradio as gr
import joblib
import pandas as pd
import numpy as np

In [4]:
# Model ve özellik dosyalarının adları
model_filename = 'xgboost_churn_model.joblib'
feature_names_filename = 'model_features.joblib'

try:
    loaded_model = joblib.load(model_filename)
    loaded_features = joblib.load(feature_names_filename)
    print("Model ve özellikler başarıyla yüklendi.")
except FileNotFoundError:
    print(f"Hata: '{model_filename}' veya '{feature_names_filename}' dosyaları bulunamadı. Lütfen aynı dizinde olduklarından emin olun.")
    # Dosyalar bulunamazsa uygulamayı durdurmak veya kullanıcıya bilgi vermek gerekebilir
    exit() # Hata durumunda script'ten çıkmak için

print(f"Yüklü Model: {loaded_model}")
print(f"Yüklü Özellikler: {loaded_features[:5]} ... (ilk 5 özellik)") # Özellik listesinin bir kısmını gösterelim

Model ve özellikler başarıyla yüklendi.
Yüklü Model: XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, feature_weights=None, gamma=None,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.1, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=3, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=100, n_jobs=None,
              num_parallel_tree=None, ...)
Yüklü Özellikler: ['Gender', 'Age', 'Under 30', 'Senior Citizen', 'Married'] ... (ilk 5 özellik)


In [ ]:
# Tahmin Fonksiyonu
def predict_churn(
    gender, senior_citizen, partner, dependents, tenure, phone_service,
    multiple_lines, internet_service, online_security, online_backup,
    device_protection, tech_support, streaming_tv, streaming_movies,
    contract, paperless_billing, payment_method, monthly_charges, total_charges
):
    # Kullanıcı girdilerini bir sözlüğe dönüştür
    input_data = {
        'Gender': gender,
        'SeniorCitizen': senior_citizen, # Gradio'dan boolean/int olarak gelebilir, dikkat
        'Partner': partner,
        'Dependents': dependents,
        'Tenure': tenure,
        'PhoneService': phone_service,
        'MultipleLines': multiple_lines,
        'InternetService': internet_service,
        'OnlineSecurity': online_security,
        'OnlineBackup': online_backup,
        'DeviceProtection': device_protection,
        'TechSupport': tech_support,
        'StreamingTV': streaming_tv,
        'StreamingMovies': streaming_movies,
        'Contract': contract,
        'PaperlessBilling': paperless_billing,
        'PaymentMethod': payment_method,
        'MonthlyCharges': monthly_charges,
        'TotalCharges': total_charges
    }

    # Tek bir satırlık DataFrame oluştur
    df_input = pd.DataFrame([input_data])

    # KATEGORİK DEĞİŞKENLER İÇİN ONE-HOT ENCODING YAPILMASI
    # Eğitimde kullandığımız kategorik sütunları burada da dönüştürmeliyiz.
    # Bunlar genellikle object veya category dtipli sütunlardı.
    # Önemli: Eğer sayısal gibi görünen 'SeniorCitizen' gibi bazı sütunlar aslında kategorikse,
    # onları da doğru şekilde ele almak gerekir. Eğitimdeki X_train.columns'ı kontrol edin.

    # Kategorik sütunları belirle (eğitimdeki kolon isimlerine göre!)
    # Bu listeyi, sizin eğitim notebook'unuzdaki X_train'in kolon isimlerine bakarak doğruca doldurmalısınız.
    # Örneğin: X_train.select_dtypes(include='object').columns.tolist()
    categorical_cols = [
        'Gender', 'Married', 'Dependents', 'Phone Service', 'Multiple Lines',
        'Internet Service', 'Online Security', 'Online Backup', 'Device Protection Plan',
        'Premium Tech Support', 'Streaming TV', 'Streaming Movies', 'Contract',
        'PaperlessBilling', 'PaymentMethod'
    ]

    # SeniorCitizen 0/1 olduğu için burada kategorik olarak ele almayalım, zaten doğru formatta.
    # df_input['SeniorCitizen'] = df_input['SeniorCitizen'].astype(int) # Gradio'dan boolean gelebilir.

    # One-Hot Encoding uygula
    df_input_encoded = pd.get_dummies(df_input, columns=categorical_cols, drop_first=False) # drop_first=False önemli, çünkü eğitimde de drop_first yapmadıysanız.

    # EKSİK KOLONLARI SIFIR İLE DOLDURMA (ÇOK ÖNEMLİ!)
    # Eğitim setindeki tüm kolonların yeni veride de aynı sırada olduğundan emin olmalıyız.
    # loaded_features, eğitimdeki kolon isimlerinin listesi.
    missing_cols = set(loaded_features) - set(df_input_encoded.columns)
    for c in missing_cols:
        df_input_encoded[c] = 0

    # FAZLA KOLONLARI KALDIRMA
    # Yeni veride, eğitimde olmayan ekstra sütunlar oluşmuşsa onları kaldır.
    extra_cols = set(df_input_encoded.columns) - set(loaded_features)
    df_input_encoded = df_input_encoded.drop(columns=list(extra_cols))

    # KOLONLARIN SIRASINI EĞİTİMDEKİ SIRAYA GÖRE AYARLAMA (ÇOK ÖNEMLİ!)
    df_final = df_input_encoded[loaded_features]

    # SAYISAL DEĞİŞKENLER İÇİN ÖLÇEKLENDİRME (EĞER EĞİTİMDE YAPTIYSANIZ)
    # Eğer eğitimde MonthlyCharges ve TotalCharges gibi sütunlar için MinMaxScaler kullandıysanız,
    # bu scaler nesnesini de kaydetmeli ve burada yükleyip kullanmalısınız.
    # Örnek:
    # try:
    #     loaded_scaler = joblib.load('minmax_scaler.joblib')
    #     df_final[['MonthlyCharges', 'TotalCharges']] = loaded_scaler.transform(df_final[['MonthlyCharges', 'TotalCharges']])
    # except FileNotFoundError:
    #     print("MinMaxScaler dosyası bulunamadı. Sayısal değerler ölçeklendirilmeyecek.")
    # Eğer eğitimde ölçeklendirme yapmadıysanız bu kısma gerek yok.
    # Current assumption: NO SCALING ON NUMERIC FEATURES

    # Tahmin yap
    prediction_proba = loaded_model.predict_proba(df_final)[0][1] # Churn olasılığı (1 sınıfının olasılığı)
    prediction_class = loaded_model.predict(df_final)[0] # 0 veya 1

    # Sonucu metin olarak formatla
    churn_status = "Müşteri Churn Edecek" if prediction_class == 1 else "Müşteri Churn Etmeyecek"
    result_text = f"{churn_status} (Churn Olasılığı: %{prediction_proba:.2f})"

    return result_text